# DM Halo Merger Remnant: Density Profile/Shape (Pre vs Post Merger)

## Goals

This project looks at the density profile and shape of the dark matter halos of MW & M31 before and after they have merged. 
The specific question I'll be answering using this code is: what shape is the halo remnant? Prolate, Oblate or Triaxial.
I will create an isodensity plot from each galaxy at snapshot 0 as well as a final density projection of the remnant. In each plot I'll overplot an ellipse that matches as closely as possible my denisty countour in order to predict the axial ratios of the halo.

In [1]:
# Need to import data from snapshot 0 and last snapshot 
# Need to create a function that rotates the angular momentum vector of my halos so that they are aligned with 
# the postive z-direction (Lab 7)
# Create density countour diagrams from MW & M31 halos and one more for the merger
# Plot an ellipse centered at zero with a given major axis and axis ratios 
# Overplot the ellipse, varying the axis ratio until your ellipse roughly matches a contour. Do this at a few different radii. 
# This gives me the axial ratio so that I can define “prolate, oblate or triaxial” (Slide 5 Lecture 15)
# """Questions:
#         How do I get the data from the last snapshot? How do I combine it into
#         a single object (the remnant) 
# """

### Plot I'm trying to reproduce
![title](./density_sample.png)


In [2]:
# Jimena Stephenson
# with code from R. Hoffman and E. Patel

# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile

In [3]:
# Code for plotting contours
#from https://gist.github.com/adrn/3993992

import scipy.optimize as so

def find_confidence_interval(x, pdf, confidence_level):
    return pdf[pdf > x].sum() - confidence_level

def density_contour(xdata, ydata, nbins_x, nbins_y, ax=None, **contour_kwargs):
    """ Create a density contour plot.
    Parameters
    ----------
    xdata : numpy.ndarray
    ydata : numpy.ndarray
    nbins_x : int
        Number of bins along x dimension
    nbins_y : int
        Number of bins along y dimension
    ax : matplotlib.Axes (optional)
        If supplied, plot the contour to this axis. Otherwise, open a new figure
    contour_kwargs : dict
        kwargs to be passed to pyplot.contour()
        
    Example Usage
    -------------
     density_contour(x pos, y pos, contour res, contour res, axis, colors for contours)
     e.g.:
     density_contour(xD, yD, 80, 80, ax=ax, colors=['red','orange', 'yellow', 'orange', 'yellow'])

    """

    H, xedges, yedges = np.histogram2d(xdata, ydata, bins=(nbins_x,nbins_y), normed=True)
    x_bin_sizes = (xedges[1:] - xedges[:-1]).reshape((1,nbins_x))
    y_bin_sizes = (yedges[1:] - yedges[:-1]).reshape((nbins_y,1))

    pdf = (H*(x_bin_sizes*y_bin_sizes))
    
    X, Y = 0.5*(xedges[1:]+xedges[:-1]), 0.5*(yedges[1:]+yedges[:-1])
    Z = pdf.T
    fmt = {}
    
    ### Adjust Here #### 
    
    # Contour Levels Definitions
    one_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.68))
    two_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.95))
    three_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.99))
    
    # You might need to add a few levels
    
    one_sigma1 = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.80))
    one_sigma2 = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.90))

    # Array of Contour levels. Adjust according to the above
    levels = [one_sigma, one_sigma1, one_sigma2, two_sigma, three_sigma][::-1]
    
    # contour level labels  Adjust accoding to the above.
    strs = ['0.68','0.80','0.90', '0.95', '0.99'][::-1]

    
    ###### 
    
    if ax == None:
        contour = plt.contour(X, Y, Z, levels=levels, origin="lower", **contour_kwargs)
        for l, s in zip(contour.levels, strs):
            fmt[l] = s
        plt.clabel(contour, contour.levels, inline=True, fmt=fmt, fontsize=12)

    else:
        contour = ax.contour(X, Y, Z, levels=levels, origin="lower", **contour_kwargs)
        for l, s in zip(contour.levels, strs):
            fmt[l] = s
        ax.clabel(contour, contour.levels, inline=True, fmt=fmt, fontsize=12)
    
    return contour

In [4]:
# Create a COM object for MW & M31 DM Using Code from CenterOfMass
COMMW = CenterOfMass("MW_000.txt",1)
COM31 = CenterOfMass("M31_000.txt",1)

#### CREATE A COM FOR THE MERGER

In [5]:
# Compute COM of MW, M31 and merger using DM particles
COMP_MW = COMMW.COM_P(0.1)
COMV_MW = COMMW.COM_V(COMP_MW[0], COMP_MW[1], COMP_MW[2])

COMP_M31 = COM31.COM_P(0.1)
COMV_M31 = COM31.COM_V(COMP_M31[0], COMP_M31[1], COMP_M31[2])

### COM_P COM_V MERGER

In [6]:
# Determine positions of disk particles relative to COM 

## MW ###
x_mw = COMMW.x - COMP_MW[0].value
y_mw = COMMW.y - COMP_MW[1].value
z_mw = COMMW.z - COMP_MW[2].value

## M31 ###
x_m31 = COM31.x - COMP_M31[0].value
y_m31 = COM31.y - COMP_M31[1].value
z_m31 = COM31.z - COMP_M31[2].value

### merger ###

# Magnitude of vector position  

## MW ##
rtot_mw = np.sqrt(x_mw**2 + y_mw**2 + z_mw**2)
## M31 ##
rtot_m31 = np.sqrt(x_m31**2 + y_m31**2 + z_m31**2)
## merger ##


# Determine velocities of DM particles for MW, M31 and merger relative to COM

## MW ###
vx_mw = COMMW.vx - COMV_MW[0].value
vy_mw = COMMW.vy - COMV_MW[1].value
vz_mw = COMMW.vz - COMV_MW[2].value

## M31 ### 
vx_m31 = COM31.vx - COMV_M31[0].value
vy_m31 = COM31.vy - COMV_M31[1].value
vz_m31 = COM31.vz - COMV_M31[2].value

## merger ###


# Magnitude of vector velocity

## MW ##
vtot = np.sqrt(vx_mw**2 + vy_mw**2 + vy_mw**2 )
## M31 ##
vtot = np.sqrt(vx_m31**2 + vy_m31**2 + vy_m31**2 )
## merger ##


# r & v vectors 

## MW ##
r_mw = np.array([x_mw, y_mw, z_mw]).T #transposed
v_mw = np.array([vx_mw, vy_mw, vz_mw]).T
## M31 ##
r_m31 = np.array([x_m31, y_m31, z_m31]).T
v_m31 = np.array([vx_m31, vy_m31, vz_m31]).T
## merger ##


In [7]:
# Rotate the position and velocity vectors so that the DM angular momentum
# is aligned with z+ axis from Lab 7

def RotateFrame(posI,velI):
    # input:  3D array of positions and velocities
    # returns: 3D array of rotated positions and velocities such that j is in z direction

    # compute the angular momentum
    L = np.sum(np.cross(posI,velI), axis=0)
    # normalize the vector
    L_norm = L/np.sqrt(np.sum(L**2))


    # Set up rotation matrix to map L_norm to z unit vector (disk in xy-plane) #### I don't have a disk
    
    # z unit vector
    z_norm = np.array([0, 0, 1])
    
    # cross product between L and z
    vv = np.cross(L_norm, z_norm)
    s = np.sqrt(np.sum(vv**2))
    
    # dot product between L and z 
    c = np.dot(L_norm, z_norm)
    
    # rotation matrix
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

    # Rotate coordinate system
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T
    
    return pos, vel

In [8]:
# Roated velocity and position vectors of MW, M31, and merger

## MW ##
rmw_new, vmw_new = RotateFrame(r_mw, v_mw)
## M31 ##
rm31_new, vm31_new = RotateFrame(r_m31, v_m31)
## merger ##

In [9]:
# Rotated MW DM - EDGE ON 

# MW DM Density 
fig, ax= plt.subplots()

# plot the particle density for MW , 2D histogram
plt.hist2d(rmw_new[:,0], vmw_new[:,2], bins=150, norm=LogNorm(), cmap='plasma')

### plot elipse

plt.colorbar()

# Add axis labels
plt.xlabel('x (kpc) ', fontsize=22)
plt.ylabel('z (kpc) ', fontsize=22)

#set axis limits
plt.ylim(-40,40)
plt.xlim(-40,40)

#adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size


# Rotated MW DM - FACE ON 

In [10]:
# Rotated M31 DM - EDGE ON 
# Rotated M31 DM - FACE ON 